In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, GRU, LSTM, SimpleRNN
from keras.callbacks import EarlyStopping

In [3]:
# load data
data = pd.read_csv('weather_prediction_dataset.csv')
features = data.drop('TOURS_temp_max', axis=1)
targets = data['TOURS_temp_max']

FileNotFoundError: [Errno 2] No such file or directory: 'weather_prediction_dataset.csv'

In [ ]:
# the row of starting day of year 2008 is 365*7+2
# the Month in row 365*7, and 365*+1 is 12 that's mean 365*7+2 is the new year after 2007
print(features['MONTH'][365*7+0])
print(features['MONTH'][365*7+1])
print(features['MONTH'][365*7+2])

In [ ]:
# normalize features
features = (features - features.mean()) / features.std()

In [ ]:
# Split the data into training and validation sets
m = 365*7+2
X_train, y_train, X_val, y_val = features[:m], targets[:m], features[m:], targets[m:]

In [ ]:
# use PCA to reduce number of features, and select the k new features with highest variance
# We will use PCA only in train data, 
# as validation data used only to detect the accuracy of the model, we shouldn't inference from it
cov_matrix = np.cov(X_train.T)
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
sort_indices = eigenvalues.argsort()[::-1]
eigenvalues = eigenvalues[sort_indices]
eigenvectors = eigenvectors[:, sort_indices]

eigenvalues

In [ ]:
# we will select only the first 7 eigenvectors with the highest variance
n_features = 7
selected_features = eigenvectors[:, :n_features]
X_train = np.dot(X_train, selected_features)
X_val = np.dot(X_val, selected_features)

In [ ]:
def generate_time_series_data(features, labels, time_step):
  n = features.shape[0]-time_step
  X = [features[i:i+time_step, :] for i in range(n)]
  Y = labels[time_step:]
  
  X = np.array(X)
  Y = np.array(Y)

  return X, Y

In [ ]:
def run_models(time_step):
  X_time_train, y_time_train = generate_time_series_data(X_train, y_train, time_step)
  X_time_val, y_time_val = generate_time_series_data(X_val, y_val, time_step)

  # Define the model RNN architecture
  model_RNN = Sequential()
  model_RNN.add(SimpleRNN(units=32, input_shape=(X_time_train.shape[1], X_time_train.shape[2])))
  model_RNN.add(Dense(units=1))

  # Define the model LSTM architecture
  model_LSTM = Sequential()
  model_LSTM.add(LSTM(units=32, input_shape=(X_time_train.shape[1], X_time_train.shape[2])))
  model_LSTM.add(Dense(units=1))

  # Define the model GRU architecture
  model_GRU = Sequential()
  model_GRU.add(GRU(units=32, input_shape=(X_time_train.shape[1], X_time_train.shape[2])))
  model_GRU.add(Dense(units=1))

  # Compile the model
  model_RNN.compile(optimizer='adam', loss='mse')
  model_LSTM.compile(optimizer='adam', loss='mse')
  model_GRU.compile(optimizer='adam', loss='mse')

  #
  early_stop_callback = EarlyStopping(
      monitor='val_loss', # Monitor validation loss
      min_delta=0.1, # Minimum change in loss to qualify as an improvement
      patience=3, # Number of epochs to wait for improvement before stopping
      verbose=1, # Print information about stopping
      mode='auto', # Automatically detect direction of improvement
      restore_best_weights=True # Restore the best model weights
  )

  # Train the model
  print("########################################## Model RNN ##########################################")
  model_RNN.fit(X_time_train, y_time_train, epochs=20, batch_size=32, validation_data=(X_time_val, y_time_val))
  print("########################################## Model LSTM ##########################################")
  model_LSTM.fit(X_time_train, y_time_train, epochs=20, batch_size=32, validation_data=(X_time_val, y_time_val))
  print("########################################## Model GRU ##########################################")
  model_GRU.fit(X_time_train, y_time_train, epochs=20, batch_size=32, validation_data=(X_time_val, y_time_val))


In [ ]:
run_models(12)